https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2

https://learn.deeplearning.ai/courses/knowledge-graphs-rag/lesson/5/constructing-a-knowledge-graph-from-text-documents

https://learn.deeplearning.ai/courses/advanced-retrieval-for-ai/lesson/1/introduction

In [41]:
from embedding_model import Embedder
import chromadb




In [40]:
embedder = Embedder(model_name='sentence-transformers/all-MiniLM-L12-v2',
                    tokenizer_name='sentence-transformers/all-MiniLM-L12-v2')

### Chroma DB

In [13]:
!pip3 show chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Name: chromadb
Version: 0.4.9
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /home/stanislav/.local/lib/python3.10/site-packages
Requires: bcrypt, chroma-hnswlib, fastapi, importlib-resources, numpy, onnxruntime, overrides, posthog, pulsar-client, pydantic, pypika, requests, tokenizers, tqdm, typing-extensions, uvicorn
Required-by: 


In [33]:


db_client = chromadb.PersistentClient(path="./persistent_storage")

collection = db_client.create_collection("my_collection")



In [34]:
docs = ["Hello World! My name is Stas",
        "Let me introduce my self, I am Stas", 
        "A white cat has eaten and now he is full",]





In [35]:
##Adding docs

collection.add(
    
    
    documents=docs,
    metadatas=[{"len" : len(docs[0]), "uri" : 232},
               {"len" : len(docs[1]), "uri" : 235},
              {"len" : len(docs[2]), "uri" : 239}],
    
    ids=["uri9", "uri10", "uri12"],
    
    embeddings=[ 
                [0.1, 0.3, 0.4],
                 [-0.1, 0.3, 0.5],
                 [2, 0, 0.03]  
                            ]
        
)

In [36]:
collection.peek(2)

{'ids': ['uri9', 'uri10'],
 'embeddings': [[0.1, 0.3, 0.4], [-0.1, 0.3, 0.5]],
 'metadatas': [{'len': 28, 'uri': 232}, {'len': 35, 'uri': 235}],
 'documents': ['Hello World! My name is Stas',
  'Let me introduce my self, I am Stas']}

In [37]:
collection.query([[2, 0.27, 0.33],],
                 n_results=2)

{'ids': [['uri12', 'uri9']],
 'distances': [[0.16290000000000004, 3.6158000000000006]],
 'metadatas': [[{'len': 40, 'uri': 239}, {'len': 28, 'uri': 232}]],
 'embeddings': None,
 'documents': [['A white cat has eaten and now he is full',
   'Hello World! My name is Stas']]}

### Rough plan 

ChromaDB + TextEmbedding + ChatGPT3 + Custom Promts and chains + Web UI (optional)



- application takes path to directory with pdfs as argument
- split docs into chunks + add metadata
- compute embeddings and put chunks to Chroma
- Create chains for Q&A that uses metada and refers to uris of docs
- put app to Docker container
- Query expansion (optional)